# Création d'une segmentation client pour Olist

-------------------------------------------------------------------------------------------------------------------------------

Problématique : identifier différents comportements des clients afin de pouvoir faire du ciblage marketing et augmenter ainsi la rentabilité de l'entreprise

Nous disposons de plusieurs informations anonymisées de la société Olist sur : 
- les commandes
- les clients
- les produits
- les catégories des produits
- des informations sur le paiment des commandes
- les vendeurs.

Il est important de noter qu'une commande peut contenir des produits de différents vendeurs.


In [1]:
#notre package de fonctionnalités
from Package import Scripts_Analyse01 as pk

import pandas as pd
import numpy as np
import missingno
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import scipy.stats
from scipy.stats import pearsonr
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from yellowbrick.features import ParallelCoordinates
from plotly.graph_objects import Layout
import jenkspy
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale
from sklearn import preprocessing
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.features import PCA as PCA_yellow
from yellowbrick.style import set_palette
# Import train_test_split function
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import SelectKBest, f_classif,mutual_info_classif
from skfeature.function.similarity_based import fisher_score
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials
 
import mlflow
from datetime import datetime

from sklearn import preprocessing
from yellowbrick.cluster import SilhouetteVisualizer
from sklearn.cluster import AgglomerativeClustering

In [2]:

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# One-hot encoding for categorical columns with get_dummies
#####encoder les colonnes catégorielles
def one_hot_encoder(df, exclude=[], nan_as_category = True):
    cols=filter(lambda x: ("_id" not in (x)) and x not in (exclude), df.columns)
    original_columns = list(cols)
    categorical_columns = [col for col in original_columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

def agg_by_group(df_test2, tab, cols, agg_name, col_index, col_by_cat):
    for col in cols:
        df_calcul=df_test2.pivot_table(values=col, index=col_index, columns=col_by_cat, aggfunc=agg_name).reset_index()
        lt_col=list(filter(lambda x: x not in (col_index), df_calcul.columns))
        lt_col=[e + "_"+str(col) for e in lt_col]
        lt_col.insert(0,col_index)
        df_calcul.columns=pd.Index([e for e in lt_col])
        df_calcul=df_calcul.fillna(0)
        tab=tab.merge(df_calcul, how='left', on=col_index)
        #display(tab)
    return tab

def difference_dates(date2, date1, new_column, df, var_i=0):    
    df2=df.copy()
    if var_i==0:
        df2[new_column] = pd.to_numeric((pd.to_datetime(df2[date2]) - pd.to_datetime(df2[date1])).dt.days)
    if var_i==1:
        df2[new_column] = pd.to_numeric((pd.to_datetime(date2) - pd.to_datetime(df2[date1])).dt.days)
    return df2

# Preprocess application_train.csv and application_test.csv
def orders(num_rows = None, nan_as_category = False):
    # Read data and merge
    ####lecture des fichiers application_train et application test
    df = pd.read_csv('./input/olist_orders_dataset.csv', nrows= num_rows)
    print("df shape: {}".format(len(df)))
    
    #calcul aggregation - add column
    df=difference_dates("order_delivered_customer_date", "order_approved_at", 'time_delivered_approved_customer', df)
    df=difference_dates("order_approved_at", "order_purchase_timestamp", 'time_approval_purchase', df)
    df=difference_dates("order_delivered_carrier_date", "order_approved_at", 'time_delivery_carrier_approved', df)
    df=difference_dates("order_delivered_customer_date", "order_delivered_carrier_date", 'time_delivery_customer_carrier', df)
    df=difference_dates("order_delivered_customer_date", "order_estimated_delivery_date", 'time_delivery_customer_estimated', df)
        
    #df["order_purchase_year_month"]=pd.to_datetime(df["order_purchase_timestamp"]).dt.strftime('%Y%m')
    df["order_purchase_timestamp"]=pd.to_datetime(df["order_purchase_timestamp"])
    df=df.drop(['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'], axis=1)
    data, data_cat = one_hot_encoder(df, ['order_purchase_timestamp', "order_id"])
    del df
    gc.collect()
    return data, data_cat


# Preprocess bureau.csv and bureau_balance.csv
def order_reviews(num_rows = None, nan_as_category = True):
    
    ####lecture des data et encodage des variable cat
    reviews_df = pd.read_csv('./input/olist_order_reviews_dataset.csv', nrows = num_rows)
    orders_df, data_cat = orders(num_rows)
    reviews= orders_df.merge(reviews_df, how='left', on='order_id')
    #bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    print("reviews shape: {}".format(len(reviews)))
    ####calcul d'agregat pour les variables

    reviews['review_comment_message_ind']=[0 if pd.isna(i.review_comment_message)==True else 1 for i in reviews.itertuples()]
    reviews['review_comment_title_ind']=[0 if pd.isna(i.review_comment_title)==True else 1 for i in reviews.itertuples()]
    
    reviews['review_created_days'] = [None if pd.isna(i.review_creation_date)==True else (pd.to_datetime(i.review_creation_date) - pd.to_datetime(i.order_purchase_timestamp)).days for i in reviews.itertuples()]
    
    reviews['review_answer_days'] = [None if pd.isna(i.review_answer_timestamp)==True else (pd.to_datetime(i.review_answer_timestamp) - pd.to_datetime(i.review_creation_date)).days for i in reviews.itertuples()]
    reviews_aggregations = {
                            'review_score': ['mean'],
                            'review_comment_message_ind': ['sum'],
                            'review_comment_title_ind': ['sum'],
                            'review_created_days': ['sum'],
                            'review_answer_days': ['sum']
                           }
    
    del reviews["review_comment_message"]
    del reviews["review_comment_title"]
    del reviews["review_creation_date"]
    del reviews["review_answer_timestamp"]
    reviews_agg = reviews.groupby('order_id').agg(reviews_aggregations)
    reviews_agg.columns = pd.Index([e[0] + "_" + e[1].lower() for e in reviews_agg.columns.tolist()])
    for col in reviews_agg.columns:
        if ("order_" in col and "order_id"!=i) or "customer_" in col:
            del reviews_agg[i]
    del reviews
    print("reviews agg shape: {}".format(len(reviews_agg)))
    gc.collect()
    return reviews_agg

# Preprocess previous_applications.csv
def order_payments(num_rows = None, nan_as_category = True):
    payments = pd.read_csv('./input/olist_order_payments_dataset.csv', nrows = num_rows)
    #Encode categoricals variables
    data, data_cat = one_hot_encoder(payments)
    #create aggregation on quantitatives variables
    paid_aggregations = {'payment_value': ['sum'],
                  'payment_installments': ['sum']}
    for col in data_cat:
        if col!="order_id":
            paid_aggregations[col] = ['sum']
    paid_agg = data.groupby("order_id").agg(paid_aggregations)
    #rename columns
    paid_agg.columns = pd.Index([e[0] + "_" + e[1].lower() for e in paid_agg.columns.tolist()])
    
    #aggregation quantitatives var by group
    payment_aggregations=agg_by_group(payments, paid_agg, ["payment_value"], "sum","order_id","payment_type" )
    del payments,data_cat
    gc.collect()
    return payment_aggregations

def product_category_translation(num_rows = None, nan_as_category = True):
    prod_transl = pd.read_csv('./input/product_category_name_translation.csv', nrows = num_rows)
    #ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    
    gc.collect()
    return prod_transl
    
# Preprocess installments_payments.csv
def products(num_rows = None, nan_as_category = True):
    products = pd.read_csv('./input/olist_products_dataset.csv', nrows = num_rows)
    cat = product_category_translation(num_rows,nan_as_category)
    #ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    products=products.merge(cat, how="left", on="product_category_name")
    del products["product_category_name"]
    data, data_cat = one_hot_encoder(products)
    data=data.fillna(0)
    del cat
    gc.collect()
    return data, data_cat

# Preprocess credit_card_balance.csv
def sellers(num_rows = None, nan_as_category = True):
    sellers = pd.read_csv('./input/olist_sellers_dataset.csv', nrows = num_rows)
    #cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    
    #del cc
    gc.collect()
    return sellers


# Preprocess POS_CASH_balance.csv
def order_items(num_rows = None, nan_as_category = True):
    items = pd.read_csv('./input/olist_order_items_dataset.csv', nrows = num_rows)
    products_obj, prods_cat=products(num_rows = None, nan_as_category = True)
    #pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    items=items.merge(products_obj, how="left", on="product_id")
    #sell=sellers(num_rows = None, nan_as_category = True)
    #items=items.merge(sell, how="left", on="seller_id")
    items_aggregations = {}
    df_num=items.select_dtypes(include=[np.float])
    for col in prods_cat:
        if ("_id" not in col) and ("_date" not in col):
              items_aggregations[col] = ['sum']
        elif col in ["product_id", "seller_id", "item_id"]:
              items_aggregations[col] = ['size']
        elif "_date" in col:
              items_aggregations[col] = ['max']
    
    for col_num in df_num:
        items_aggregations[col_num] = ['mean']
    items_agg = items.groupby("order_id").agg(items_aggregations)
    items_agg.columns = pd.Index([e[0] + "_" + e[1] for e in items_agg.columns.tolist()])
    del items, products_obj
    gc.collect()
    
    return items_agg


def orders_informations(num_rows = None, nan_as_category = True):
    with timer("Orders "):
        df, data_cat = orders(num_rows)
        print("Order df shape:", df.shape)
        gc.collect()
    with timer("Orders reviews"):
        reviews = order_reviews(num_rows)
        print("Reviews df shape:", reviews.shape)
        df = df.merge(reviews, how='left', on='order_id')
        del reviews
        print("df reviews", df.shape)
        gc.collect()
    with timer("Process order payments"):
        payments = order_payments(num_rows)
        print("Order payments df shape:", payments.shape)
        df = df.merge(payments, how='left', on='order_id')
        del payments
        gc.collect()
        print("df payment", df.shape)
    
    with timer("Process order items"):
        items = order_items(num_rows)
        print("Order items df shape:", items.shape)
        df = df.merge(items, how='left', on='order_id')
        del items
        gc.collect()  
    return df


# Preprocess credit_card_balance.csv
def customer(num_rows = None, nan_as_category = True):
    customer_obj = pd.read_csv('./input/olist_customers_dataset.csv', nrows = num_rows)
    #del cc
    gc.collect()
    return customer_obj

# Il y a des doublons dans cette table > une ville ou un code zip a différentes coordonnées
def geolocation(num_rows = None, nan_as_category = True):
    geoloc = pd.read_csv('./input/olist_geolocation_dataset.csv', nrows = num_rows)    
    #del cc
    gc.collect()
    return geoloc

In [3]:
# lance chaque fonction avec un timer
def main(debug = True):
    num_rows = 1000 if debug else None
    
    ####import fichiers + calcul agregats (new features) et jointure de toutes les tables
    ### + nettoyage memoire
    with timer("Orders informations"):
        df = orders_informations(num_rows)
        print("orders informations shape:", df.shape)
        gc.collect()
    
    with timer("Process customer"):
        customer_obj = customer(num_rows)
        customer_obj = customer_obj.merge(df, how='left', on='customer_id')
        print("Customer df shape:", customer_obj.shape)
        del df
        gc.collect()
        customer_obj.columns = customer_obj.columns.str.strip()    
        customer_obj.columns = customer_obj.columns.str.replace(' ', '_')
        customer_obj.columns = customer_obj.columns.str.replace(r"[^a-zA-Z\d\_]+", "")  
        customer_obj.columns = customer_obj.columns.str.replace(r"[^a-zA-Z\d\_]+", "")
        
        
        ##############a modifier
        ##Moyenne - median by customer
        #count nb commande + nb produit mean + nb prod by category en moy par commande + time moyen
        customer_aggregations={}
        cols=list(customer_obj.columns)
        cols_float=customer_obj.select_dtypes(include=[np.float])
        col_ok=[]       
        
        
        for col in cols:
            if (("_sum" in col) and ("payment_" not in col)) or ('order_status' in col) or ('order_purchase' in col) :
                customer_aggregations[col] = ['sum']
                col_ok.append(col)
            if ("payment_" in col) or "_size" in col:
                customer_aggregations[col] = ['sum', 'mean']
                col_ok.append(col)
            if "order_id" in col or "customer_id" in col:
                customer_aggregations[col] = ["size"]
                col_ok.append(col)
            if ("time_" in col) or ("price_") in col or ("review_" in col) or ("product" in col and "category" not in col) or ("freight" in col):
                customer_aggregations[col] = ['mean']
                col_ok.append(col)
            if 'order_purchase_timestamp' in col:
                customer_aggregations[col] = ['max']
                col_ok.append(col)
        for col in cols_float:
            if col not in col_ok :
                customer_aggregations[col] = ['mean', 'size']
        customer_agg = customer_obj.groupby(["customer_unique_id"]).agg(customer_aggregations)
        customer_agg.columns = pd.Index([e[0] + "_" + e[1].lower() for e in customer_agg.columns.tolist()])
        
        date_max_purchase=customer_agg['order_purchase_timestamp_max'].max()
        customer_agg=difference_dates(date_max_purchase, "order_purchase_timestamp_max", "order_purchase_timestamp_recency", customer_agg,1)
        
        del customer_obj
        gc.collect()
        return customer_agg.reset_index()
    
if __name__ == "__main__":
    #submission_file_name = "submission_kernel02.csv"
    with timer("Full complete run"):
        df = main(False)
        #, feat_importance
       

df shape: 99441
Order df shape: (99441, 17)
Orders  - done in 2s
df shape: 99441
reviews shape: 99992
reviews agg shape: 99441
Reviews df shape: (99441, 5)
df reviews (99441, 22)
Orders reviews - done in 33s
Order payments df shape: (99440, 14)
df payment (99441, 35)
Process order payments - done in 1s
Order items df shape: (98666, 82)
Process order items - done in 2s
orders informations shape: (99441, 117)
Orders informations - done in 38s
Customer df shape: (99441, 121)
Process customer - done in 5s
Full complete run - done in 43s


# A SUPPRIMER

In [4]:
pk.data_uniqueone_string(df)
for i in df.columns:
    if "product_" in i:
        df[i]=df[i].fillna(0)

for i in df.columns:
    if ("payment_" in i) or ("price_" in i) or ("freight_" in i)  or ("review_" in i):
        df[i]=df[i].fillna(0)

for i in df.columns:
    if ("time_" in i):
        df[i]=df[i].fillna(max(df[i]))

del df["customer_id_size"]

#build 4 clusters for recency and add it to dataframe
kmeans = KMeans(n_clusters=5)
kmeans.fit(df[['order_purchase_timestamp_recency']])
df['RecencyCluster'] = kmeans.predict(df.loc[:,['order_purchase_timestamp_recency']])

#function for ordering cluster numbers
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

df_new = order_cluster('RecencyCluster', 'order_purchase_timestamp_recency',df,False)




customer_unique_id - count unique : 96096
customer_id_size - count unique : 9
order_id_size - count unique : 9
order_purchase_timestamp_max - count unique : 95834
time_delivered_approved_customer_mean - count unique : 251
time_approval_purchase_mean - count unique : 37
time_delivery_carrier_approved_mean - count unique : 128
time_delivery_customer_carrier_mean - count unique : 248
time_delivery_customer_estimated_mean - count unique : 331
order_status_approved_sum - count unique : 2
order_status_canceled_sum - count unique : 4
order_status_created_sum - count unique : 2
order_status_delivered_sum - count unique : 10
order_status_invoiced_sum - count unique : 2
order_status_processing_sum - count unique : 2
order_status_shipped_sum - count unique : 3
order_status_unavailable_sum - count unique : 3
deleted unique colonne : order_status_nan_sum
review_score_mean_mean - count unique : 34
review_comment_message_ind_sum_mean - count unique : 17
review_comment_title_ind_sum_mean - count uniqu

In [5]:
#k-means
kmeans = KMeans(n_clusters=3)
kmeans.fit(df_new[['order_id_size']])
df_new['FrequencyCluster'] = kmeans.predict(df_new[['order_id_size']])

#order the frequency cluster
df_new = order_cluster('FrequencyCluster', 'order_id_size',df_new,True)

#see details of each cluster
df_new.groupby('FrequencyCluster')['order_id_size'].describe()

,count,mean,std,min,25%,50%,75%,max
FrequencyCluster,,,,,,,,
0,93099.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
1,2745.0,2.000000,0.000000,2.0,2.0,2.0,2.0,2.0
2,252.0,3.380952,1.196656,3.0,3.0,3.0,3.0,17.0


In [6]:
#apply clustering
kmeans = KMeans(n_clusters=4)
kmeans.fit(df_new[['payment_value_sum_sum']])
df_new['RevenueCluster'] = kmeans.predict(df_new[['payment_value_sum_sum']])


#order the cluster numbers
df_new = order_cluster('RevenueCluster', 'payment_value_sum_sum',df_new,True)

#show details of the dataframe
df_new.groupby('RevenueCluster')['payment_value_sum_sum'].describe()

,count,mean,std,min,25%,50%,75%,max
RevenueCluster,,,,,,,,
0,75389.0,94.121296,47.147486,0.00,55.00,86.15,129.59,200.28
1,17189.0,306.151060,93.918776,200.29,229.24,277.86,359.62,585.70
2,3093.0,865.050592,247.542870,585.91,664.92,788.71,1006.87,1575.05
3,425.0,2294.422800,1057.554535,1583.71,1760.75,2026.54,2384.50,13664.08


In [13]:
metrics_history=[]

#order_purchase_timestamp_max
def model_analyse(date1, n_days, file, var_time):
    #entrainement m0
    file0=file0.loc[file[var_time]<=date_1]
    
    #standardise
    std_scale0 = preprocessing.StandardScaler().fit(file0)
    file0_std0 = std_scale0.transform(file0)
    
    #entrainement du m0
    kmeans_m0 = KMeans(n_clusters=4)
    C0 = kmeans_mo.fit(file0_std0)
    
    #file 1
    end_date = pd.to_datetime(date_1) + datetime.timedelta(days=n_days)
    file1=file1.loc[(file[var_time]>date1) & (file[var_time]<end_date)]
    
    #entrainement m1
    kmeans_m1 = KMeans(n_clusters=4)
    std_scale1 = preprocessing.StandardScaler().fit(file1)
    file1_std1 = std_scale1.transform(file1)
    C1_new = kmeans_m1.fit(file1_std1)
    
    #predict file 1 avec m0
    file1_std0 = std_scale0.transform(file1)
    C1_init = kmeans_m0.predict(file1_std0)
    
    #calcul ARI
    ARI=adjusted_rand_score(C1_init, C1_new)
    return ARI

def find_best_periode_update_model(date1, n_days, file, var_time):
    num_model=1
    date_end=max(file[var_time])
    date1=pd.to_datetime(date1)
    while date1<=date_end:
        ARI = model_analyse(date1, n_days, file, var_time)
        
        num_model=num_model+1
        date1 = pd.to_datetime(date_1) + datetime.timedelta(days=n_days)
        metrics_history.append({'model':num_model, 'n_days':n_days, 'date1':date1, 'ARI':ARI})

In [12]:
min(df_new["order_purchase_timestamp_max"])

Timestamp('2018-10-17 17:30:18')